<a href="https://colab.research.google.com/github/mmfara/Agile-final-project/blob/main/MPA_COMPAS_Re_AD_ROC___EO_PP_Model_Re%2B(EO_and_ROC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing necessary libraries

In [1]:
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]

In [2]:
!pip install BlackBoxAuditing

In [3]:
!pip install aif360[infairness]

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
# Import necessary libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.metrics import ClassificationMetric

/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.11/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

#Re + AL + ROC

In [6]:
data = pd.read_csv("/content/g_recidivism_all_group_combined_race_sex.csv")

In [7]:
data.shape

(5278, 9)

In [8]:
data.head()

,c_charge_degree,score_text,priors_count,decile_score,two_year_recid,juv_fel_count,juv_misd_count,juv_other_count,group
0,1,0,0,3,1,0,0,0,3
1,1,0,4,4,1,0,0,1,3
2,1,1,14,6,1,0,0,0,2
3,0,0,0,1,0,0,0,0,0
4,1,0,0,4,0,0,0,0,2


In [9]:
X = data.drop(['two_year_recid'], axis =1)
y = data[['two_year_recid']]

In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [11]:
print(X_train.shape)
print(X_test.shape)

(3694, 8)
(1584, 8)


In [12]:
X_train["group"].value_counts()

,count
group,
3,1837
2,1140
1,391
0,326


In [13]:
# Create training DataFrame
train_df = X_train.copy()
train_df['two_year_recid'] = y_train

test_df = X_test.copy()
test_df['two_year_recid'] = y_test

In [14]:
# Check value counts by group and label
group_label_counts = train_df.groupby(['group', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts)


       Negative (0)  Positive (1)
group                            
0               212           114
1               250           141
2               681           459
3               813          1024


In [15]:
# Check value counts by group and label
group_label_counts_test = test_df.groupby(['group', 'two_year_recid']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts_test.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts_test)


       Negative (0)  Positive (1)
group                            
0               100            56
1                96            62
2               288           193
3               355           434


In [16]:
from sklearn.utils import resample

# Oversample within training set only
balanced_parts = []

for group_val in train_df['group'].unique():
    group_data = train_df[train_df['group'] == group_val].copy()

    class_counts = group_data['two_year_recid'].value_counts()

    if len(class_counts) < 2:
        print(f"Skipping group {group_val} — only one class present.")
        continue

    maj_class = class_counts.idxmax()
    min_class = class_counts.idxmin()
    n_to_match = class_counts.max()

    maj_df = group_data[group_data['two_year_recid'] == maj_class]
    min_df = group_data[group_data['two_year_recid'] == min_class]

    min_df_upsampled = resample(min_df, replace=True, n_samples=n_to_match, random_state=42)

    balanced_group = pd.concat([maj_df, min_df_upsampled])
    balanced_parts.append(balanced_group)

# Combine all groups
balanced_df = pd.concat(balanced_parts).reset_index(drop=True)

# Show final group-label distribution
print("Balanced group-label distribution:")
print(balanced_df.groupby(['group', 'two_year_recid']).size().unstack(fill_value=0))


Balanced group-label distribution:
two_year_recid     0     1
group                     
0                212   212
1                250   250
2                681   681
3               1024  1024


In [17]:
train_df.shape

(3694, 9)

In [18]:
balanced_df.head()


,c_charge_degree,score_text,priors_count,decile_score,juv_fel_count,juv_misd_count,juv_other_count,group,two_year_recid
0,0,1,0,5,0,0,0,2,0
1,1,0,2,1,0,0,0,2,0
2,1,2,10,9,0,0,0,2,0
3,1,0,0,1,0,0,0,2,0
4,1,0,2,2,0,0,0,2,0


In [19]:
balanced_df['two_year_recid'].value_counts()

,count
two_year_recid,
0,2167
1,2167


In [20]:
balanced_df.shape

(4334, 9)

In [21]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=101, shuffle =True, stratify = y)

#Concatenation
Concatenating the X_train and y_train

Also, the concatenation of X_test and y_test

In [22]:
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [23]:
dataset_train = dataset_train.reset_index(drop=True)
dataset_test = dataset_test.reset_index(drop=True)

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [25]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [26]:
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [27]:
dataset_train_scaled = pd.concat([X_train_scaled, y_train], axis=1)

In [28]:
dataset_test_scaled = pd.concat([X_test_scaled, y_test], axis=1)

#Checking my datasets

In [29]:
dataset_train

,c_charge_degree,score_text,priors_count,decile_score,juv_fel_count,juv_misd_count,juv_other_count,group,two_year_recid
0,1,0,2,1,0,0,0,2,0
1,1,0,1,1,0,0,0,1,0
2,1,0,1,3,0,0,0,2,0
3,1,1,0,6,0,0,0,3,1
4,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
3689,0,0,1,4,0,0,0,0,0
3690,1,0,1,4,0,0,0,1,0
3691,0,0,0,1,0,0,0,2,0
3692,0,0,0,3,0,0,0,0,0


#Reweighing

In [30]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

privileged_groups = [{'group': 2}]


unprivileged_groups = [{'group': g} for g in data['group'].unique() if g != 2]

In [31]:
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

In [32]:
# Apply Reweighing
from aif360.algorithms.preprocessing import Reweighing
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)


In [33]:
# from aif360.algorithms.preprocessing import DisparateImpactRemover
# dir = DisparateImpactRemover(repair_level=1)

##Creating a binarylabeldataset

In [34]:
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['two_year_recid'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=['group'],
                             privileged_protected_attributes=privileged_groups)

In [35]:
dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['two_year_recid'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=['group'],
                                      privileged_protected_attributes=privileged_groups)

###Fitting RW on the dataset

In [36]:
# Fit and transform the training data
dir_dataset = RW.fit_transform(dataset_train_bld)

#Adversarial Debiasing

In [37]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [38]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [39]:
#New code

import numpy as np

try:
    import tensorflow.compat.v1 as tf
except ImportError as error:
    from logging import warning
    warning("{}: AdversarialDebiasing will be unavailable. To install, run:\n"
            "pip install 'aif360[AdversarialDebiasing]'".format(error))

from aif360.algorithms import Transformer

class AdversarialDebiasing(Transformer):
    def __init__(self,
                 unprivileged_groups,
                 privileged_groups,
                 scope_name,
                 sess,
                 seed=None,
                 adversary_loss_weight=0.1,
                 num_epochs=50,
                 batch_size=128,
                 classifier_num_hidden_units=200,
                 debias=True):

        super(AdversarialDebiasing, self).__init__(
            unprivileged_groups=unprivileged_groups,
            privileged_groups=privileged_groups)

        self.scope_name = scope_name
        self.seed = seed

        self.unprivileged_groups = unprivileged_groups
        self.privileged_groups = privileged_groups

        # Allow multiple groups, but ensure same protected attribute
        self.protected_attribute_name = list(self.unprivileged_groups[0].keys())[0]

        self.sess = sess
        self.adversary_loss_weight = adversary_loss_weight
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.classifier_num_hidden_units = classifier_num_hidden_units
        self.debias = debias

        self.features_dim = None
        self.features_ph = None
        self.protected_attributes_ph = None
        self.true_labels_ph = None
        self.pred_labels = None

    def _classifier_model(self, features, features_dim, keep_prob):
        with tf.variable_scope("classifier_model"):
            W1 = tf.get_variable('W1', [features_dim, self.classifier_num_hidden_units],
                                  initializer=tf.initializers.glorot_uniform(seed=self.seed1))
            b1 = tf.Variable(tf.zeros(shape=[self.classifier_num_hidden_units]), name='b1')
            h1 = tf.nn.relu(tf.matmul(features, W1) + b1)
            h1 = tf.nn.dropout(h1, keep_prob=keep_prob, seed=self.seed2)
            W2 = tf.get_variable('W2', [self.classifier_num_hidden_units, 1],
                                 initializer=tf.initializers.glorot_uniform(seed=self.seed3))
            b2 = tf.Variable(tf.zeros(shape=[1]), name='b2')
            pred_logit = tf.matmul(h1, W2) + b2
            pred_label = tf.sigmoid(pred_logit)
        return pred_label, pred_logit

    def _adversary_model(self, pred_logits, true_labels):
        with tf.variable_scope("adversary_model"):
            c = tf.get_variable('c', initializer=tf.constant(1.0))
            s = tf.sigmoid((1 + tf.abs(c)) * pred_logits)
            W = tf.get_variable('W_adv', [3, self.num_prot_classes],
                                initializer=tf.initializers.glorot_uniform(seed=self.seed4))
            b = tf.Variable(tf.zeros(shape=[self.num_prot_classes]), name='b_adv')
            logits = tf.matmul(tf.concat([s, s * true_labels, s * (1.0 - true_labels)], axis=1), W) + b
            pred_protected_attribute_label = tf.nn.softmax(logits)
        return pred_protected_attribute_label, logits

    def fit(self, dataset):
        if tf.executing_eagerly():
            raise RuntimeError("AdversarialDebiasing does not work in eager "
                               "execution mode. To fix, add `tf.disable_eager_execution()`"
                               " to the top of the calling script.")

        if self.seed is not None:
            np.random.seed(self.seed)
        ii32 = np.iinfo(np.int32)
        self.seed1, self.seed2, self.seed3, self.seed4 = np.random.randint(ii32.min, ii32.max, size=4)

        temp_labels = dataset.labels.copy()
        temp_labels[(dataset.labels == dataset.favorable_label).ravel(), 0] = 1.0
        temp_labels[(dataset.labels == dataset.unfavorable_label).ravel(), 0] = 0.0

        self.num_prot_classes = int(np.max(dataset.protected_attributes[:, dataset.protected_attribute_names.index(self.protected_attribute_name)]) + 1)

        with tf.variable_scope(self.scope_name):
            num_train_samples, self.features_dim = np.shape(dataset.features)

            self.features_ph = tf.placeholder(tf.float32, shape=[None, self.features_dim])
            self.protected_attributes_ph = tf.placeholder(tf.int32, shape=[None])
            self.true_labels_ph = tf.placeholder(tf.float32, shape=[None, 1])
            self.keep_prob = tf.placeholder(tf.float32)

            self.pred_labels, pred_logits = self._classifier_model(self.features_ph, self.features_dim, self.keep_prob)
            pred_labels_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.true_labels_ph, logits=pred_logits))

            if self.debias:
                pred_protected_attributes_labels, pred_protected_attributes_logits = self._adversary_model(pred_logits, self.true_labels_ph)
                pred_protected_attributes_loss = tf.reduce_mean(
                    tf.nn.sparse_softmax_cross_entropy_with_logits(
                        labels=self.protected_attributes_ph,
                        logits=pred_protected_attributes_logits))

            global_step = tf.Variable(0, trainable=False)
            starter_learning_rate = 0.001
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       1000, 0.96, staircase=True)
            classifier_opt = tf.train.AdamOptimizer(learning_rate)
            if self.debias:
                adversary_opt = tf.train.AdamOptimizer(learning_rate)

            classifier_vars = [var for var in tf.trainable_variables(scope=self.scope_name) if 'classifier_model' in var.name]
            if self.debias:
                adversary_vars = [var for var in tf.trainable_variables(scope=self.scope_name) if 'adversary_model' in var.name]
                adversary_grads = {var: grad for (grad, var) in adversary_opt.compute_gradients(pred_protected_attributes_loss, var_list=classifier_vars)}

            normalize = lambda x: x / (tf.norm(x) + np.finfo(np.float32).tiny)

            classifier_grads = []
            for (grad, var) in classifier_opt.compute_gradients(pred_labels_loss, var_list=classifier_vars):
                if self.debias:
                    unit_adversary_grad = normalize(adversary_grads[var])
                    grad -= tf.reduce_sum(grad * unit_adversary_grad) * unit_adversary_grad
                    grad -= self.adversary_loss_weight * adversary_grads[var]
                classifier_grads.append((grad, var))
            classifier_minimizer = classifier_opt.apply_gradients(classifier_grads, global_step=global_step)

            if self.debias:
                with tf.control_dependencies([classifier_minimizer]):
                    adversary_minimizer = adversary_opt.minimize(pred_protected_attributes_loss, var_list=adversary_vars)

            self.sess.run(tf.global_variables_initializer())
            self.sess.run(tf.local_variables_initializer())

            for epoch in range(self.num_epochs):
                shuffled_ids = np.random.choice(num_train_samples, num_train_samples, replace=False)
                for i in range(num_train_samples // self.batch_size):
                    batch_ids = shuffled_ids[self.batch_size * i: self.batch_size * (i + 1)]
                    batch_features = dataset.features[batch_ids]
                    batch_labels = np.reshape(temp_labels[batch_ids], [-1, 1])
                    batch_protected_attributes = dataset.protected_attributes[batch_ids][:, dataset.protected_attribute_names.index(self.protected_attribute_name)].astype(int)

                    batch_feed_dict = {self.features_ph: batch_features,
                                       self.true_labels_ph: batch_labels,
                                       self.protected_attributes_ph: batch_protected_attributes,
                                       self.keep_prob: 0.8}
                    if self.debias:
                        _, _, pred_labels_loss_value, pred_protected_attributes_loss_vale = self.sess.run(
                            [classifier_minimizer, adversary_minimizer, pred_labels_loss, pred_protected_attributes_loss],
                            feed_dict=batch_feed_dict)
                        if i % 200 == 0:
                            print("epoch %d; iter: %d; batch classifier loss: %f; batch adversarial loss: %f" % (
                                epoch, i, pred_labels_loss_value, pred_protected_attributes_loss_vale))
                    else:
                        _, pred_labels_loss_value = self.sess.run(
                            [classifier_minimizer, pred_labels_loss], feed_dict=batch_feed_dict)
                        if i % 200 == 0:
                            print("epoch %d; iter: %d; batch classifier loss: %f" % (
                                epoch, i, pred_labels_loss_value))
        return self

    def predict(self, dataset):
        if self.seed is not None:
            np.random.seed(self.seed)

        num_test_samples, _ = np.shape(dataset.features)

        samples_covered = 0
        pred_labels = []
        while samples_covered < num_test_samples:
            start = samples_covered
            end = samples_covered + self.batch_size
            if end > num_test_samples:
                end = num_test_samples
            batch_ids = np.arange(start, end)
            batch_features = dataset.features[batch_ids]
            batch_labels = np.reshape(dataset.labels[batch_ids], [-1, 1])
            batch_protected_attributes = dataset.protected_attributes[batch_ids][:, dataset.protected_attribute_names.index(self.protected_attribute_name)].astype(int)

            batch_feed_dict = {self.features_ph: batch_features,
                               self.true_labels_ph: batch_labels,
                               self.protected_attributes_ph: batch_protected_attributes,
                               self.keep_prob: 1.0}

            pred_labels += self.sess.run(self.pred_labels, feed_dict=batch_feed_dict)[:, 0].tolist()
            samples_covered += len(batch_features)

        dataset_new = dataset.copy(deepcopy=True)
        dataset_new.scores = np.array(pred_labels, dtype=np.float64).reshape(-1, 1)
        dataset_new.labels = (np.array(pred_labels) > 0.5).astype(np.float64).reshape(-1, 1)

        temp_labels = dataset_new.labels.copy()
        temp_labels[(dataset_new.labels == 1.0).ravel(), 0] = dataset.favorable_label
        temp_labels[(dataset_new.labels == 0.0).ravel(), 0] = dataset.unfavorable_label
        dataset_new.labels = temp_labels.copy()

        return dataset_new


In [40]:
#New code
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

sess = tf.Session()

# Assuming you have a 'group' column as a multi-valued protected attribute
adv = AdversarialDebiasing(
    privileged_groups=[{'group': 2}],           # or whichever group is privileged
    unprivileged_groups=[{'group': g} for g in data['group'].unique() if g != 2],         # dummy placeholder (still needed)
    scope_name='adv_debias_group2',
    sess=sess,
    debias=True,
    seed=101
)

adv.fit(dataset_train_bld)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.685774; batch adversarial loss: 1.432027
epoch 1; iter: 0; batch classifier loss: 0.688732; batch adversarial loss: 1.417445
epoch 2; iter: 0; batch classifier loss: 0.686337; batch adversarial loss: 1.348742
epoch 3; iter: 0; batch classifier loss: 0.643110; batch adversarial loss: 1.335585
epoch 4; iter: 0; batch classifier loss: 0.607151; batch adversarial loss: 1.291708
epoch 5; iter: 0; batch classifier loss: 0.640284; batch adversarial loss: 1.278594
epoch 6; iter: 0; batch classifier loss: 0.592142; batch adversarial loss: 1.305632
epoch 7; iter: 0; batch classifier loss: 0.632164; batch adversarial loss: 1.295794
epoch 8; iter: 0; batch classifier loss: 0.583483; batch adversarial loss: 1.237856
epoch 9; iter: 0; batch classifier loss: 0.581946; batch adversarial loss: 1.221235
epoch 10; iter: 0; batch classifier loss: 0.607831; batch adversarial loss: 1.212938
epoch 11; iter: 0; batch classifier loss: 0.592657; batch adversarial loss:

In [41]:
predictions = adv.predict(dataset_test_bld)

In [42]:
#new code
import pandas as pd
import matplotlib.pyplot as plt
from aif360.metrics import ClassificationMetric

# Make sure your predictions and test_bld are already defined
# Replace 'group' with your protected attribute name if needed
protected_attr = 'group'
privileged_group_val = 2  # Change this if your privileged group is different

# Get list of unique groups from the BinaryLabelDataset
group_idx = dataset_test_bld.feature_names.index(protected_attr)
group_values = sorted(np.unique(dataset_test_bld.features[:, group_idx]))

# Collect metrics for each group compared to the privileged group
results = []

for group_val in group_values:
    if group_val == privileged_group_val:
        continue  # skip comparing privileged group to itself

    metric = ClassificationMetric(
        dataset_test_bld,
        predictions,
        unprivileged_groups=[{protected_attr: group_val}],
        privileged_groups=[{protected_attr: privileged_group_val}]
    )

    results.append({
        'group': group_val,
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        "Predictive Equality Difference": metric.false_positive_rate_difference(),
        'Accuracy': metric.accuracy()
    })

# Convert to DataFrame for plotting and inspection
df_results = pd.DataFrame(results)
display(df_results)

# # Plot SPD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Statistical Parity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Statistical Parity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('SPD')
# plt.grid(True)
# plt.show()

# # Plot EOD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Equal Opportunity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Equal Opportunity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('EOD')
# plt.grid(True)
# plt.show()


,group,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Accuracy
0,0.0,-0.001529,0.006583,0.994765,0.028788,0.656566
1,1.0,0.142995,0.204663,1.489593,0.146764,0.656566
2,3.0,0.144532,0.134706,1.494853,0.074827,0.656566


In [43]:
from aif360.metrics import ClassificationMetric

# Assuming dataset_test_bld is the original test dataset with true labels
# and predictions is the dataset with your model's predictions
metric_predicted_dataset = ClassificationMetric(dataset_test_bld, predictions,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)

accuracy = metric_predicted_dataset.accuracy()
disparate_impact = metric_predicted_dataset.disparate_impact()
# Add any other metrics you are interested in

print(f"Accuracy: {accuracy}")
print(f"Disparate Impact: {disparate_impact}")
# Print other metrics


Accuracy: 0.6565656565656566
Disparate Impact: 1.424728613367925


In [44]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_predicted_dataset.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_predicted_dataset.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_predicted_dataset.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.656566
Disparate Impact: 1.424729
Mean Difference: 0.124050


In [45]:
# Print out the metrics
print(f"Statistical Parity Difference: {metric_predicted_dataset.statistical_parity_difference()}")
print(f"Disparate Impact: {metric_predicted_dataset.disparate_impact()}")
print(f"Equal Opportunity Difference: {metric_predicted_dataset.equal_opportunity_difference()}")
print(f"Predictive Equality: {metric_predicted_dataset.false_positive_rate_difference()}")

Statistical Parity Difference: 0.12405033001655064
Disparate Impact: 1.424728613367925
Equal Opportunity Difference: 0.1295733836056417
Predictive Equality: 0.08011750154607297


#ROC

In [46]:
# Apply Reject Option Classification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, metric_name="Equal opportunity difference",
                                 low_class_thresh=0.3, high_class_thresh=0.8, num_class_thresh=100, num_ROC_margin=50,
                                 metric_ub=0.05, metric_lb=-0.05)

In [47]:
test_predictions = ROC.fit_predict(dataset_test_bld, predictions)

In [48]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [49]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.643308
Disparate Impact: 1.096428
Mean Difference: 0.061736


In [50]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [51]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: 0.061736026640611064
Disparate Impact: 1.09642757031177
Equal Opportunity Difference: 0.03578410836475354


In [52]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = 0.035784
Difference in False Positive Rates (Unprivileged - Privileged) = 0.041020


In [53]:
#new code
import pandas as pd
import matplotlib.pyplot as plt
from aif360.metrics import ClassificationMetric

# Make sure your predictions and test_bld are already defined
# Replace 'group' with your protected attribute name if needed
protected_attr = 'group'
privileged_group_val = 2  # Change this if your privileged group is different

# Get list of unique groups from the BinaryLabelDataset
group_idx = dataset_test_bld.feature_names.index(protected_attr)
group_values = sorted(np.unique(dataset_test_bld.features[:, group_idx]))

# Collect metrics for each group compared to the privileged group
results = []

for group_val in group_values:
    if group_val == privileged_group_val:
        continue  # skip comparing privileged group to itself

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[{protected_attr: group_val}],
        privileged_groups=[{protected_attr: privileged_group_val}]
    )

    results.append({
        'group': group_val,
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        "Predictive Equality Difference": metric.false_positive_rate_difference(),
        'Accuracy': metric.accuracy()
    })

# Convert to DataFrame for plotting and inspection
df_results = pd.DataFrame(results)
display(df_results)

# # Plot SPD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Statistical Parity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Statistical Parity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('SPD')
# plt.grid(True)
# plt.show()

# # Plot EOD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Equal Opportunity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Equal Opportunity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('EOD')
# plt.grid(True)
# plt.show()


,group,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Accuracy
0,0.0,-0.160502,-0.167215,0.749306,-0.116061,0.643308
1,1.0,0.015612,0.101292,1.024385,0.014272,0.643308
2,3.0,0.114016,0.051219,1.178086,0.095341,0.643308


#EO

In [54]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

In [55]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Initialize EqualizedOdds
eo = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, seed = 101)

# Fit the EqualizedOdds model
test_predictions = eo.fit_predict(dataset_test_bld, predictions)

In [56]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [57]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.644571
Disparate Impact: 1.063616
Mean Difference: 0.020180


In [58]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [59]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: 0.020179864005264336
Disparate Impact: 1.0636157907970833
Equal Opportunity Difference: 0.01183493925429413


In [60]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = 0.011835
Difference in False Positive Rates (Unprivileged - Privileged) = -0.009629


In [61]:
#new code
import pandas as pd
import matplotlib.pyplot as plt
from aif360.metrics import ClassificationMetric

# Make sure your predictions and test_bld are already defined
# Replace 'group' with your protected attribute name if needed
protected_attr = 'group'
privileged_group_val = 2  # Change this if your privileged group is different

# Get list of unique groups from the BinaryLabelDataset
group_idx = dataset_test_bld.feature_names.index(protected_attr)
group_values = sorted(np.unique(dataset_test_bld.features[:, group_idx]))

# Collect metrics for each group compared to the privileged group
results = []

for group_val in group_values:
    if group_val == privileged_group_val:
        continue  # skip comparing privileged group to itself

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[{protected_attr: group_val}],
        privileged_groups=[{protected_attr: privileged_group_val}]
    )

    results.append({
        'group': group_val,
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        "Predictive Equality Difference": metric.false_positive_rate_difference(),
        'Accuracy': metric.accuracy()
    })

# Convert to DataFrame for plotting and inspection
df_results = pd.DataFrame(results)
display(df_results)

# # Plot SPD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Statistical Parity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Statistical Parity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('SPD')
# plt.grid(True)
# plt.show()

# # Plot EOD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Equal Opportunity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Equal Opportunity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('EOD')
# plt.grid(True)
# plt.show()


,group,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Accuracy
0,0.0,-0.073971,-0.118499,0.766809,-0.018081,0.644571
1,1.0,0.059409,0.063974,1.187282,0.090971,0.644571
2,3.0,0.030498,0.020544,1.096142,-0.037893,0.644571


#STARTING OFF FROM PRE(Re ABOVE) -> POST(ROC AND EO)

In [62]:
dir_dataset

               instance weights        features                          \
                                                                          
                                c_charge_degree score_text priors_count   
instance names                                                            
0                      0.873806             1.0        0.0          2.0   
1                      1.065597             1.0        0.0          1.0   
2                      0.873806             1.0        0.0          1.0   
3                      0.935208             1.0        1.0          0.0   
4                      1.065597             0.0        0.0          0.0   
...                         ...             ...        ...          ...   
3689                   1.065597             0.0        0.0          1.0   
3690                   1.065597             1.0        0.0          1.0   
3691                   0.873806             0.0        0.0          0.0   
3692                   1.

In [63]:
# Define features and target variable
X_train = dir_dataset.features
y_train = dir_dataset.labels.ravel()
X_test = dataset_test_bld.features
y_test = dataset_test_bld.labels.ravel()

In [64]:
X_train

array([[ 1.,  0.,  2., ...,  0.,  0.,  2.],
       [ 1.,  0.,  1., ...,  0.,  0.,  1.],
       [ 1.,  0.,  1., ...,  0.,  0.,  2.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  2., 18., ...,  1.,  0.,  3.]])

In [65]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [66]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)

X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

y_train = pd.DataFrame(y_train, columns = y.columns)

y_test = pd.DataFrame(y_test, columns = y.columns)

In [67]:
dataset_train = pd.concat([X_train_scaled, y_train], axis=1)

dataset_test = pd.concat([X_test_scaled, y_test], axis=1)

#Model

In [68]:
model = LogisticRegression()

In [69]:
model.fit(X_train_scaled, y_train['two_year_recid'])

LogisticRegression()

In [70]:
y_pred = model.predict(X_test_scaled)

In [71]:
accuracy_score(y_pred, y_test)

0.6755050505050505

In [72]:
y_pred

array([1., 1., 0., ..., 0., 1., 1.])

In [73]:
y_pred = pd.DataFrame(y_pred, columns=['two_year_recid'])

In [74]:
X_test = pd.DataFrame(X_test, columns=X.columns)

In [75]:
dataset_test = pd.concat([X_test, y_test], axis=1)

dataset_testpred = pd.concat([X_test, y_pred], axis=1)

In [76]:
# Convert DataFrames back to AIF360 BinaryLabelDataset
dataset_test = BinaryLabelDataset(df=dataset_test,
                                          label_names=['two_year_recid'],
                                          favorable_label=favorable_label,
                                          unfavorable_label=unfavorable_label,
                                          protected_attribute_names=['group'],
                                          privileged_protected_attributes=privileged_groups)

dataset_testpred = BinaryLabelDataset(df=dataset_testpred,
                                         label_names=['two_year_recid'],
                                         favorable_label=favorable_label,
                                         unfavorable_label=unfavorable_label,
                                         protected_attribute_names=['group'],
                                         privileged_protected_attributes=privileged_groups)

#Re+EO

In [77]:
from aif360.algorithms.postprocessing import EqOddsPostprocessing
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Initialize EqualizedOdds
eo = EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, seed = 101)

# Fit the EqualizedOdds model
test_predictions = eo.fit_predict(dataset_test, dataset_testpred)

In [78]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [79]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.648359
Disparate Impact: 1.067168
Mean Difference: 0.021177


In [80]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [81]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: 0.021176892859279373
Disparate Impact: 1.0671684270444628
Equal Opportunity Difference: -0.003883884932271986


In [82]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.003884
Difference in False Positive Rates (Unprivileged - Privileged) = 0.003704


In [83]:
#new code
import pandas as pd
import matplotlib.pyplot as plt
from aif360.metrics import ClassificationMetric

# Make sure your predictions and test_bld are already defined
# Replace 'group' with your protected attribute name if needed
protected_attr = 'group'
privileged_group_val = 2  # Change this if your privileged group is different

# Get list of unique groups from the BinaryLabelDataset
group_idx = dataset_test_bld.feature_names.index(protected_attr)
group_values = sorted(np.unique(dataset_test_bld.features[:, group_idx]))

# Collect metrics for each group compared to the privileged group
results = []

for group_val in group_values:
    if group_val == privileged_group_val:
        continue  # skip comparing privileged group to itself

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[{protected_attr: group_val}],
        privileged_groups=[{protected_attr: privileged_group_val}]
    )

    results.append({
        'group': group_val,
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        "Predictive Equality Difference": metric.false_positive_rate_difference(),
        'Accuracy': metric.accuracy()
    })

# Convert to DataFrame for plotting and inspection
df_results = pd.DataFrame(results)
display(df_results)

# # Plot SPD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Statistical Parity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Statistical Parity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('SPD')
# plt.grid(True)
# plt.show()

# # Plot EOD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Equal Opportunity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Equal Opportunity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('EOD')
# plt.grid(True)
# plt.show()


,group,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Accuracy
0,0.0,-0.213929,-0.275181,0.321464,-0.146162,0.648359
1,1.0,-0.107488,-0.106714,0.659071,-0.070453,0.648359
2,3.0,0.092563,0.039429,1.293588,0.070396,0.648359


#Re+ROC

In [84]:
# Apply Reject Option Classification
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups, metric_name="Equal opportunity difference",
                                 low_class_thresh=0.3, high_class_thresh=0.8, num_class_thresh=100, num_ROC_margin=50,
                                 metric_ub=0.05, metric_lb=-0.05)

In [85]:
test_predictions = ROC.fit_predict(dataset_test, dataset_testpred)

/usr/local/lib/python3.11/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


In [86]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test_bld,
    test_predictions,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

In [87]:
print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.675505
Disparate Impact: 1.664131
Mean Difference: 0.186265


In [88]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [89]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: 0.18626493050708887
Disparate Impact: 1.6641308211873445
Equal Opportunity Difference: 0.209939254294093


In [90]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = 0.209939
Difference in False Positive Rates (Unprivileged - Privileged) = 0.122758


In [91]:
#new code
import pandas as pd
import matplotlib.pyplot as plt
from aif360.metrics import ClassificationMetric

# Make sure your predictions and test_bld are already defined
# Replace 'group' with your protected attribute name if needed
protected_attr = 'group'
privileged_group_val = 2  # Change this if your privileged group is different

# Get list of unique groups from the BinaryLabelDataset
group_idx = dataset_test_bld.feature_names.index(protected_attr)
group_values = sorted(np.unique(dataset_test_bld.features[:, group_idx]))

# Collect metrics for each group compared to the privileged group
results = []

for group_val in group_values:
    if group_val == privileged_group_val:
        continue  # skip comparing privileged group to itself

    metric = ClassificationMetric(
        dataset_test_bld,
        test_predictions,
        unprivileged_groups=[{protected_attr: group_val}],
        privileged_groups=[{protected_attr: privileged_group_val}]
    )

    results.append({
        'group': group_val,
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        "Predictive Equality Difference": metric.false_positive_rate_difference(),
        'Accuracy': metric.accuracy()
    })

# Convert to DataFrame for plotting and inspection
df_results = pd.DataFrame(results)
display(df_results)

# # Plot SPD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Statistical Parity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Statistical Parity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('SPD')
# plt.grid(True)
# plt.show()

# # Plot EOD
# plt.figure(figsize=(10, 6))
# plt.bar(df_results['group'], df_results['Equal Opportunity Difference'])
# plt.axhline(0, color='black', linestyle='--')
# plt.title('Equal Opportunity Difference by Group')
# plt.xlabel('Group')
# plt.ylabel('EOD')
# plt.grid(True)
# plt.show()


,group,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Accuracy
0,0.0,-0.138572,-0.152074,0.505918,-0.095960,0.675505
1,1.0,-0.033711,0.013192,0.879803,-0.021036,0.675505
2,3.0,0.293392,0.274829,2.046094,0.230959,0.675505
